In [161]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_colwidth', 20)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
app = pd.read_excel('sample_data/Mandrill.xlsx', sheet_name='AboutMandrill')
other = pd.read_excel('sample_data/Mandrill.xlsx', sheet_name='AboutOther')
test = pd.read_excel('sample_data/Mandrill.xlsx', sheet_name='TestTweets')
test['Tweet'] = test['Tweet'].str.lower()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
def clean_text(text):
  # 1. Tokenize the text:
  tokens = nltk.word_tokenize(text)

  # 2. Remove unwanted characters (optional):
  # Define the characters you want to remove
  unwanted_chars = "[^a-zA-Z0-9\s]"  # Remove anything except letters, numbers, and spaces
  tokens = [char for char in tokens if char not in unwanted_chars]

  # 3. Remove stop words (optional):
  stop_words = nltk.corpus.stopwords.words('english')  # Download stopwords if needed
  filtered_tokens = [token for token in tokens if token not in stop_words]

  # 4. Join the cleaned tokens back into a string (optional):
  clean_text = " ".join(filtered_tokens)

  return filtered_tokens


In [5]:
app['tokens'] = app['Tweet'].apply(clean_text)
other['tokens'] = other['Tweet'].apply(clean_text)
test['tokens'] = test['Tweet'].apply(clean_text)

In [6]:
app.head()

,Tweet,tokens
0,[blog] Using Nullmailer and Mandrill for your ...,"[blog, Using, Nullmailer, Mandrill, Ubuntu, Li..."
1,[blog] Using Postfix and free Mandrill email s...,"[blog, Using, Postfix, free, Mandrill, email, ..."
2,@aalbertson There are several reasons emails g...,"[@, aalbertson, There, several, reasons, email..."
3,@adrienneleigh I just switched it over to Mand...,"[@, adrienneleigh, I, switched, Mandrill, ,, l..."
4,@ankeshk +1 to @mailchimp We use MailChimp for...,"[@, ankeshk, +1, @, mailchimp, We, use, MailCh..."


In [8]:
# gathering all the tokens in a list
app_token_list = []
for tokens in app['tokens']:
  for token in tokens:
    app_token_list.append(token)

other_token_list = []
for tokens in other['tokens']:
  for token in tokens:
    other_token_list.append(token)

In [9]:
print(len(app_token_list))
print(len(other_token_list))

2478
2107


In [21]:
app_token_dict = {}
other_token_dict = {}

# counting words
for word in app_token_list:
    if word not in app_token_dict:
        app_token_dict[word] = 0
    app_token_dict[word] += 1

for word in other_token_list:
    if word not in other_token_dict:
        other_token_dict[word] = 0
    other_token_dict[word] += 1

In [24]:
app_token_dict_df = pd.DataFrame.from_dict(app_token_dict, orient="index")
other_token_dict_df = pd.DataFrame.from_dict(other_token_dict, orient="index")

In [36]:
# renaming columns
app_token_dict_df.index.names = ['token']
app_token_dict_df.reset_index(inplace=True)
app_token_dict_df.rename(columns={0:'count'}, inplace=True)

In [55]:
# renaming columns
other_token_dict_df.index.names = ['token']
other_token_dict_df.reset_index(inplace=True)
other_token_dict_df.rename(columns={0:'count'}, inplace=True)

In [57]:
# adding one
app_token_dict_df['add_one'] = app_token_dict_df['count'] + 1
other_token_dict_df['add_one'] = other_token_dict_df['count'] + 1

In [61]:
# calculating the probability
app_token_dict_df['probability'] = app_token_dict_df['add_one'] / len(app_token_list)
other_token_dict_df['probability'] = other_token_dict_df['add_one'] / len(other_token_list)

In [63]:
# calculating the log of p
app_token_dict_df['ln(p)'] = np.log(app_token_dict_df['probability'])
other_token_dict_df['ln(p)'] = np.log(other_token_dict_df['probability'])

In [82]:
# lower casing the tokens
app_token_dict_df['token'] = app_token_dict_df['token'].str.lower()
other_token_dict_df['token'] = other_token_dict_df['token'].str.lower()

In [67]:
other_token_dict_df.head()

,token,count,add_one,probability,ln(p)
0,¿En,1,2,0.000949,-6.959873
1,donde,1,2,0.000949,-6.959873
2,esta,2,3,0.001424,-6.554408
3,su,1,2,0.000949,-6.959873
4,remontada,1,2,0.000949,-6.959873


In [70]:
np.log(1/len(app_token_list))

-7.815207062189088

In [109]:
if app_token_dict_df['token'].eq('just11').any():
  print('yes')

In [132]:
# looking for token the app dict

for i,tokens in enumerate(test['tokens']):
  app_dcit = {}
  for token in tokens:
    if app_token_dict_df['token'].eq(token).any():
      app_dcit[token] = app_token_dict_df.loc[app_token_dict_df['token'] == token, 'ln(p)'].tolist()[0]
    else:
      app_dcit[token] = np.log(1/len(app_token_list))
  test['app_probability'][i] = app_dcit

app_dcit = {}
for i,tokens in enumerate(test['tokens']):
  app_dcit = {}
  for token in tokens:
    if other_token_dict_df['token'].eq(token).any():
      app_dcit[token] = other_token_dict_df.loc[other_token_dict_df['token'] == token, 'ln(p)'].tolist()[0]
    else:
      app_dcit[token] = np.log(1/len(app_token_list))
  test['other_probability'][i] = app_dcit

<ipython-input-132-2701f18190fb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['app_probability'][i] = app_dcit
<ipython-input-132-2701f18190fb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['other_probability'][i] = app_dcit


In [146]:
def count_val(dict_):
  total = 0
  for k in dict_:
    total += dict_[k]
  return total

In [148]:
test['app_probability_sum'] = test['app_probability'].apply(count_val)
test['other_probability_sum'] = test['other_probability'].apply(count_val)

In [156]:


test['classification'] = np.where(test['app_probability_sum'] > test['other_probability_sum'], 'app', 'other')




In [163]:
test.to_excel('classification.xlsx')